# Operating LWRs
The whole goal of this notebook is to demonstrate the method used to pull the LWR deployment information from the EIA.

In [1]:
import numpy as np
import pandas as pd
import sys
import os
from collections import defaultdict
import textwrap

In [2]:
import scenario_definitions as sd

In [3]:
# %%
# the source of the LWR information for the csv is this eia table, only run if
# you don't have the csv
table = pd.read_html('https://www.eia.gov/nuclear/spent_fuel/ussnftab2.php')
long = table[0].to_dict()
long_df = pd.DataFrame(long)

long_df.to_csv('lwr_info.csv')

In [4]:
long_df

,Reactor name,State,Reactor type,Reactor vendora,Core size (number of assemblies),Startup date (year) b,License expiration (year),Actual retirement (year)
0,Arkansas Nuclear One 1,AR,PWR,B&W,177,1974,2034,NaN
1,Arkansas Nuclear One 2,AR,PWR,CE,177,1978,2038,NaN
2,Beaver Valley 1,PA,PWR,WE,157,1976,2036,NaN
3,Beaver Valley 2,PA,PWR,WE,157,1987,2047,NaN
4,Big Rock Point,MI,BWR,GE,84,1964,NaN,1997
...,...,...,...,...,...,...,...,...
115,Wolf Creek 1,KS,PWR,WE,193,1985,2045,NaN
116,Yankee Rowe,MA,PWR,WE,76,1960,NaN,1991
117,Zion 1,IL,PWR,WE,193,1973,NaN,1997
118,Zion 2,IL,PWR,WE,193,1973,NaN,1996


In [5]:
def generate_facility_xml(df, reactor):
    """
    Generate the XML string for a reactor facility from the given dataframe.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing the information about the reactor.
    reactor : str
        The name of reactor to generate the XML for.

    Returns
    -------
    str
        The XML string for the reactor facility.

    Notes
    -----
    * This function assumes that LWRs will all operate for 80 years unless they are prematurely retired.
    * The user must confirm the latitude, longitude, and power capacity of each reactor.
    """
    # Find the index of the reactor in the dataframe
    reactor_index = df[df['Reactor name'] == reactor].index[0]

    # Extract the information about the reactor
    reactor_name = df['Reactor name'].iloc[reactor_index]
    startup_date = df['Startup date (year) b'].iloc[reactor_index]
    retirement_date = df['Actual retirement (year)'].iloc[reactor_index]
    core_size = df['Core size (number of assemblies)'].iloc[reactor_index]

    # If retirement_date is NaN, set it to 80 years after the startup date
    if pd.isna(retirement_date):
        retirement_date = int(startup_date) + 80
    else:
        retirement_date = int(retirement_date)

    # Calculate the lifetime of the reactor in months
    life_months = str((retirement_date - int(startup_date)) * 12)

    # Format the information into the desired XML structure
    xml_string = textwrap.dedent(f"""
<facility>
  <name>{reactor_name}</name>
  <lifetime>{life_months}</lifetime>
  <config>
    <Reactor>
      <fuel_incommods>  <val>fresh_uox</val> </fuel_incommods>
      <fuel_inrecipes>  <val>fresh_uox</val> </fuel_inrecipes>
      <fuel_outcommods> <val>used_uox</val> </fuel_outcommods>
      <fuel_outrecipes> <val>used_uox</val> </fuel_outrecipes>
      <cycle_time>18</cycle_time>
      <refuel_time>1</refuel_time>
      <assem_size>427.38589211618256</assem_size>
      <n_assem_core>{core_size}</n_assem_core>
      <n_assem_batch>80</n_assem_batch>
      <power_cap></power_cap>
      <longitude></longitude>
      <latitude></latitude>
    </Reactor>
  </config>
</facility>
""").strip()
    return xml_string

## An example of this function in action

In [6]:
generate_facility_xml(long_df, 'Arkansas Nuclear One 1')

'<facility>\n  <name>Arkansas Nuclear One 1</name>\n  <lifetime>960</lifetime>\n  <config>\n    <Reactor>\n      <fuel_incommods>  <val>fresh_uox</val> </fuel_incommods>\n      <fuel_inrecipes>  <val>fresh_uox</val> </fuel_inrecipes>\n      <fuel_outcommods> <val>used_uox</val> </fuel_outcommods>\n      <fuel_outrecipes> <val>used_uox</val> </fuel_outrecipes>\n      <cycle_time>18</cycle_time>\n      <refuel_time>1</refuel_time>\n      <assem_size>427.38589211618256</assem_size>\n      <n_assem_core>177</n_assem_core>\n      <n_assem_batch>80</n_assem_batch>\n      <power_cap></power_cap>\n      <longitude></longitude>\n      <latitude></latitude>\n    </Reactor>\n  </config>\n</facility>'

## Double check values with symbols
In the database, there are several reactors that have special characters next to their names. For our purposes, these symbols will be removed. I identified these through trial and error, I'm sure there was a more pythonic way to do that.

In [7]:
print(long_df.loc[33,'Reactor name'],',',long_df.loc[33,'Actual retirement (year)'])
long_df.loc[33,'Actual retirement (year)'] = '2020'

print(long_df.loc[48,'Reactor name'],',',long_df.loc[48,'Actual retirement (year)'])

long_df.loc[48,'Actual retirement (year)'] = '2020'

print(long_df.loc[78,'Reactor name'],',',long_df.loc[78,'Actual retirement (year)'])

long_df.loc[78,'Actual retirement (year)'] = '2019'

print(long_df.loc[105,'Reactor name'],',',long_df.loc[105,'Actual retirement (year)'])

long_df.loc[105,'Actual retirement (year)'] = '2019'

Duane Arnold , 2020*
Indian Point 2 , 2020*
Pilgrim 1 , 2019*
Three Mile Island 1 , 2019*


## Now we will generate all of the LWR files for the simulations

In [8]:
# specify the location these .xml files should be saved to relative to this one
output_dir = '../reactors/lwrs/'

# if the output directory does not exist, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [9]:
# iterate through each reactor in the dataframe and generate an .xml file for it
# we exclude the last one, because that row is a source, not a reactor.
for reactor in long_df['Reactor name'].tolist()[0:-1]:
    xml_string = generate_facility_xml(long_df, reactor)
    with open(f'{output_dir}{reactor}.xml', 'w') as f:
        f.write(xml_string)